In [1]:
%load_ext autoreload
%autoreload 2

import asyncio
import os
from typing import *

import torch
import wandb
from spot.data import get_tk_dataset_name
from spot.function_dataset import data_project_from_dir
from spot.model import ModelWrapper
from spot.train import TrainingConfig, PreprocessArgs
from spot.type_env import AccuracyMetric
from spot.utils import (
    PickleCache,
    assert_eq,
    get_dataroot,
    get_dataset_dir,
    get_eval_dir,
    get_gpu_id,
    get_model_dir,
    pickle_dump,
    pmap,
    pretty_print_dict,
    pretty_show_dict,
    proj_root,
    run_long_task,
    write_file,
)
from spot.visualization import string_to_html
from termcolor import colored

os.chdir(proj_root())


def wandb_string(s: str):
    return wandb.Html(string_to_html(s))

/home/jiayi/Projects/SPOT/.venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# experiment configurations
quicktest = False

gpu_id = get_gpu_id(1)
# model_name = "model-v6--TrainingConfig(func_only=False, left_margin=2048, preamble_size=800, right_margin=1536)"
model_name = "model-v6--TrainingConfig(func_only=False, imports_in_preamble=False, stub_in_preamble=False, left_margin=2048, right_margin=1536)"
pre_args = PreprocessArgs(imports_in_preamble=False, stub_in_preamble=False)
# dataset_name = "ManyTypes4Py"
dataset_name = "InferTypes4Py"
# dataset_name = "SPOT-src"
experiment_name = dataset_name + ": " + model_name

print(colored(f"Use GPU: {gpu_id}", "green"))


GPU_ID not set, using: 1
Use GPU: 1


In [3]:
# load test data
from spot.data import load_tokenized_srcsets, create_tokenized_srcsets

sdata_name = get_tk_dataset_name(dataset_name, pre_args, func_only=False)
sdata_path = get_dataroot() / "TokenizedSrcSets" / sdata_name
recreate=False
if recreate or not sdata_path.exists():
    create_tokenized_srcsets(
        dataset_name,
        sdata_path,
        func_only=False,
        pre_args=pre_args,
    )
tk_dataset = load_tokenized_srcsets(
    sdata_path,
    quicktest=quicktest,
    sets_to_load=["test"],
)


Loading TokenizedSrcSets:  /mnt/nas/jiayi/SPOT/TokenizedSrcSets/InferTypes4Py-v5-PreprocessArgs(imports_in_preamble=False, stub_in_preamble=False)
2.4M	/mnt/nas/jiayi/SPOT/TokenizedSrcSets/InferTypes4Py-v5-PreprocessArgs(imports_in_preamble=False, stub_in_preamble=False)


In [4]:
# model evaluation

from spot.function_decoding import (
    DecodingOrders,
    EvalResult,
    PreprocessArgs,
    RolloutCtx,
)
from spot.function_dataset import sigmap_from_file_predictions
from spot.static_analysis import SignatureErrorAnalysis

# load model
model = ModelWrapper.from_pretrained(get_model_dir() / model_name)
device = torch.device(f"cuda:{gpu_id}" if torch.cuda.is_available() else "cpu")
model.to(device)

ctx_args = model.args.ctx_args
model.args.sampling_max_tokens = ctx_args.ctx_size
model.args.do_sample = False
model.args.num_beams = 10
model.args.tokens_per_type = 16

eval_cache = PickleCache(get_eval_dir(dataset_name, model_name) / f"{pre_args}")
# eval_cache.clear()
pre_r = eval_cache.cached(
    "DatasetPredResult.pkl",
    lambda: model.eval_on_dataset(tk_dataset["test"]),
)


In [5]:
repos_dir = get_dataset_dir(dataset_name) / "repos" / "test"
test_repo_paths = [f for f in repos_dir.iterdir() if f.is_dir()]
test_projects = pmap(
    data_project_from_dir,
    test_repo_paths,
    desc="Loading test projects",
)
assert len(test_projects) > 0

common_names = ModelWrapper.load_common_type_names(get_model_dir() / model_name)
pred_map, label_map = sigmap_from_file_predictions(pre_r, test_projects, repos_dir)
accs = {
    m.name: SignatureErrorAnalysis(pred_map, label_map, m).accuracies
    for m in AccuracyMetric.default_metrics(common_names)
}

pretty_print_dict(accs)

Loading test projects: 100%|██████████| 3/3 [00:10<00:00,  3.63s/it]


plain_acc:
   plain_acc: 68.80% (count=2.7k)
   plain_acc_by_common:
      rare: 51.64% (count=916)
      common: 77.83% (count=1.7k)
   plain_acc_by_simple:
      complex: 54.35% (count=863)
      simple: 75.75% (count=1.8k)
   plain_acc_by_cat:
      FuncArg: 68.73% (count=1.8k)
      FuncReturn: 72.62% (count=650)
      ClassAtribute: 58.61% (count=244)
      GlobalVar: 100.00% (count=4)
   plain_acc_label_size: 1.7192
   plain_acc_pred_size: 1.6417
   plain_acc_ignored_labels: 0
   n_missing_types: 2
acc:
   acc: 68.95% (count=2.5k)
   acc_by_common:
      rare: 53.28% (count=916)
      common: 78.06% (count=1.6k)
   acc_by_simple:
      complex: 54.04% (count=731)
      simple: 75.14% (count=1.8k)
   acc_by_cat:
      FuncArg: 68.72% (count=1.6k)
      FuncReturn: 71.45% (count=613)
      ClassAtribute: 64.46% (count=242)
      GlobalVar: 0.00% (count=1)
   acc_label_size: 1.6482
   acc_pred_size: 1.5636
   acc_ignored_labels: 164
   n_missing_types: 2
base_acc:
   base_acc: 77.78

In [16]:
from spot.utils import decode_tokens, Path
from spot.visualization import export_preds_on_code

export_to = Path(f"caches/model_predictions/eval_file_model/{dataset_name}")
export_preds_on_code(pre_r.chunks, pre_r.predictions, export_to, AccuracyMetric(common_names))

Computing accuracies: 100%|██████████| 1851/1851 [00:00<00:00, 12425.55it/s]
